In [2]:
import polars as pl

# Overview of PySpark data management

In this notebook, we will illustrate how various data verbs are implemented in `pyspark`.

## Why `polars`?

The main advantages of using `polars` over `pandas` are

1. Lazy evaluation and column expression,
2. Parallel processing out-of-the-box,
3. Dot-chained queries, and
4. Data verbs related to `SQL` and/or `dplyr`.

## Data verbs in `polars`

In this lecture, we will review the common data verbs are implemented in `polars`.  Luckily, the implementation of `polars` is inspired by `SQL`, which should be familiar from DSCI 325 and/or CS 385.

### Overview of Basic Data Verbs in `polars`

Verb/Function           | `polars`                            |
------------------------|-------------------------------------|
Column expression       | `pl.col('name') ...`                |
Literal expression      | `pl.lit(value)`                     |
SELECT                  | `.select(...)`                      |
FILTER                  | `.filter(...)`                      |
MUTATE                  | `.with_columns(...)`                |
SIMPLE AGGREGATION      | `.select(pl.mean(...), ...)`        |
GROUPBY                 | `.group_by(...)`                    |
AGGREGATE               | `.agg(...)`                         |
JOIN                    | `left_tbl.join(right_tbl,...)`      |
UNION                   | `pl.concat` or SQL                  |
STACK COLUMNS           | `.unpivot(...)`                     |
UNSTACK COLUMNS         | `.pivot(...)`                       |

In [158]:
(heroes := 
 pl.read_csv('./data/heroes_information.csv',
            infer_schema_length=10000,
            )
).head()

ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,i64
0,"""A-Bomb""","""Male""","""yellow""","""Human""","""No Hair""",203.0,"""Marvel Comics""","""-""","""good""",441
1,"""Abe Sapien""","""Male""","""blue""","""Icthyo Sapien""","""No Hair""",191.0,"""Dark Horse Comics""","""blue""","""good""",65
2,"""Abin Sur""","""Male""","""blue""","""Ungaran""","""No Hair""",185.0,"""DC Comics""","""red""","""good""",90
3,"""Abomination""","""Male""","""green""","""Human / Radiation""","""No Hair""",203.0,"""Marvel Comics""","""-""","""bad""",441
4,"""Abraxas""","""Male""","""blue""","""Cosmic Entity""","""Black""",-99.0,"""Marvel Comics""","""-""","""bad""",-99


## Selecting Columns

The first verb, `select` 

* filters the *columns*
* At the core of `SQL` statements

In [164]:
(select_query :=
 heroes
 .select(heroes['name'],      # Column via dataframe['name'] BAD~
         pl.col('Gender'),    # Column expression (lazy) GOOD!
         'Weight',
        )                     # String GOOD!
).head()

name,Gender,Weight
str,str,i64
"""A-Bomb""","""Male""",441
"""Abe Sapien""","""Male""",65
"""Abin Sur""","""Male""",90
"""Abomination""","""Male""",441
"""Abraxas""","""Male""",-99


## Filtering Rows

The next verb, `filter` 

* filters the *rows*
* is related to the `SQL` `WHERE` clause
* `polars`: Use the `filter` method

In [161]:
pl.col('Gender') == 'Male' # <-- Lazy column expression

<Expr ['[(col("Gender")) == (String(Ma…'] at 0x16851E780>

In [165]:
(heroes
 .filter(pl.col('Gender') == 'Male')
).head()

ID,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
i64,str,str,str,str,str,f64,str,str,str,i64
0,"""A-Bomb""","""Male""","""yellow""","""Human""","""No Hair""",203.0,"""Marvel Comics""","""-""","""good""",441
1,"""Abe Sapien""","""Male""","""blue""","""Icthyo Sapien""","""No Hair""",191.0,"""Dark Horse Comics""","""blue""","""good""",65
2,"""Abin Sur""","""Male""","""blue""","""Ungaran""","""No Hair""",185.0,"""DC Comics""","""red""","""good""",90
3,"""Abomination""","""Male""","""green""","""Human / Radiation""","""No Hair""",203.0,"""Marvel Comics""","""-""","""bad""",441
4,"""Abraxas""","""Male""","""blue""","""Cosmic Entity""","""Black""",-99.0,"""Marvel Comics""","""-""","""bad""",-99


## Chaining Data Verbs

* Processing df $\rightarrow$ chaining data verbs
* Accomplished through dot-chains

In [166]:
(heroes
 .filter(pl.col('Gender') == 'Male')
 .select('name', 
         'Gender', 
         'Weight')
).head()

name,Gender,Weight
str,str,i64
"""A-Bomb""","""Male""",441
"""Abe Sapien""","""Male""",65
"""Abin Sur""","""Male""",90
"""Abomination""","""Male""",441
"""Abraxas""","""Male""",-99


## Constructing New Columns

The third verb, `mutate` 

* Creates new columns
* Changes existing columns
* `polars`: Use the `with_columns` method

### Example 3 - Converting Weight to kilograms

In [167]:
# Lazy expression
pl.col('Weight')/2.2046

<Expr ['[(col("Weight")) / (dyn float:…'] at 0x1687D0DA0>

In [169]:
# Using keyword assignment [preferred]
(heroes
 .select('name', 
         'Gender', 
         'Weight')
 .with_columns(Weight_kg = pl.col('Weight')/2.2046)  # LHS must be a python name
).head()

name,Gender,Weight,Weight_kg
str,str,i64,f64
"""A-Bomb""","""Male""",441,200.036288
"""Abe Sapien""","""Male""",65,29.483807
"""Abin Sur""","""Male""",90,40.823732
"""Abomination""","""Male""",441,200.036288
"""Abraxas""","""Male""",-99,-44.906105


In [170]:
# Using .alias [not as readable]
(heroes
 .select('name', 
         'Gender', 
         'Weight')
 .with_columns((pl.col('Weight')/2.2046).alias('Weight (kg)')  # Alias allows non-python names
              )
).head()

name,Gender,Weight,Weight (kg)
str,str,i64,f64
"""A-Bomb""","""Male""",441,200.036288
"""Abe Sapien""","""Male""",65,29.483807
"""Abin Sur""","""Male""",90,40.823732
"""Abomination""","""Male""",441,200.036288
"""Abraxas""","""Male""",-99,-44.906105


## Referencing a new column

 Use the `col` function with the label from `withColumn`

In [171]:
(new_col_result := 
 heroes
 .select('name', 
         'Gender', 
         'Weight',
        )
 .with_columns(Weight_kg =  pl.col('Weight')/2.2046)
 .filter(pl.col('Weight_kg') < 100)  # <-- one reason we need lazy expressions
).head()

name,Gender,Weight,Weight_kg
str,str,i64,f64
"""Abe Sapien""","""Male""",65,29.483807
"""Abin Sur""","""Male""",90,40.823732
"""Abraxas""","""Male""",-99,-44.906105
"""Absorbing Man""","""Male""",122,55.338837
"""Adam Monroe""","""Male""",-99,-44.906105


## Simple and Grouped Aggregation

In `polars` we use
- `select` with summary functions for SIMPLE AGGREGATION, and
- `group_by` + `agg` for grouped aggregation.

In [172]:
(pitching :=  
 pl.read_csv('data/baseball/core/Pitching.csv', 
            )
).head()

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,f64,str,i64,str,i64,i64,i64,i64,str,str,str
"""bechtge01""",1871,1,"""PH1""","""NA""",1,2,3,3,2,0,0,78,43,23,0,11,1,null,7.96,null,7,null,0,146,0,42,null,null,null
"""brainas01""",1871,1,"""WS3""","""NA""",12,15,30,30,30,0,0,792,361,132,4,37,13,null,4.5,null,7,null,0,1291,0,292,null,null,null
"""fergubo01""",1871,1,"""NY2""","""NA""",0,0,1,0,0,0,0,3,8,3,0,0,0,null,27.0,null,2,null,0,14,0,9,null,null,null
"""fishech01""",1871,1,"""RC1""","""NA""",4,16,24,24,22,1,0,639,295,103,3,31,15,null,4.35,null,20,null,0,1080,1,257,null,null,null
"""fleetfr01""",1871,1,"""NY2""","""NA""",0,1,1,1,1,0,0,27,20,10,0,3,0,null,10.0,null,0,null,0,57,0,21,null,null,null


### Simple Aggregation

A **simple aggregation** collapses all rows into one row.

<img src="https://github.com/wsu-stat489/module5_intro_to_pyspark/blob/main/img/simple_aggregation.png?raw=1" width=800>

In [129]:
(pitching
  .select(pl.mean('ERA').alias('Mean(ERA)'),
          pl.std('ERA').alias('SD(ERA)'),
          pl.max('W').alias('Max(Wins)'),
          pl.min('W').alias('Min(Wins)'))
)

Mean(ERA),SD(ERA),Max(Wins),Min(Wins)
f64,f64,i64,i64
5.11792,5.466654,60,0


### Group and Aggregate

<img src="https://github.com/wsu-stat489/module5_intro_to_pyspark/blob/main/img/group_and_aggregate.png?raw=1" width=800>

In [173]:
(eras := 
 pitching
 .filter((pl.col('yearID') >= 1900) & (pl.col('yearID') < 1940)) 
 .with_columns(era = (pl.when(pl.col('yearID') < 1920)
                        .then(pl.lit("dead ball")) 
                        .otherwise(pl.lit("after dead ball")) 
                     )
              )
 .group_by('era')
 .agg(mean_runs = pl.mean('R'))
)

era,mean_runs
str,f64
"""after dead ball""",55.289698
"""dead ball""",49.707696


### Grouping by more than one category

* `group_by` accepts multiple columns
* Groups all combinations

In [174]:
(pitching
 .select('yearID', 'teamID', 'W')
 .filter(pl.col('yearID') >= 1946)
 .group_by('yearID', 'teamID')
 .agg(total_wins = pl.sum('W'))
 .filter(pl.col('total_wins') >= 100)
 .sort(['yearID',
        'total_wins'],
       descending = [False, 
                     True]
      )
)

yearID,teamID,total_wins
i64,str,i64
1946,"""BOS""",104
1953,"""BRO""",105
1954,"""CLE""",111
1954,"""NYA""",103
1961,"""NYA""",109
…,…,…
2018,"""NYA""",100
2019,"""HOU""",107
2019,"""LAN""",106


## Joins in `polars`

Performed with `df_left.join(df_right, how=type_str)`

In [175]:
(dept := 
 pl.read_csv("./data/department.csv",  
            )
)

DeptID,DeptName
i64,str
31,"""Sales"""
33,"""Engineering"""
34,"""Clerical"""
35,"""Marketing"""


In [176]:
(empl := 
 pl.read_csv("./data/employee.csv",  
            )
)

LastName,DeptID
str,i64
"""Rafferty""",31
"""Jones""",33
"""Heisenberg""",33
"""Robinson""",34
"""Smith""",34
"""Williams""",null


### `on` VS. `left_on` & `right_on`

- Use `on` when the key column(s) have the same name.

#### `on` Example

In [177]:
(empl
 .join(dept, 
       on = 'DeptID',  # Use a list if 2+ keys
      )
)

LastName,DeptID,DeptName
str,i64,str
"""Rafferty""",31,"""Sales"""
"""Jones""",33,"""Engineering"""
"""Heisenberg""",33,"""Engineering"""
"""Robinson""",34,"""Clerical"""
"""Smith""",34,"""Clerical"""


#### `left_on` + `right_on` Example

In [178]:
(empl2 :=
 empl.rename({'DeptID':'department',
             })
)

LastName,department
str,i64
"""Rafferty""",31
"""Jones""",33
"""Heisenberg""",33
"""Robinson""",34
"""Smith""",34
"""Williams""",null


In [180]:
(empl2
 .join(dept, 
       left_on = 'department',  # Use a list if 2+ keys
       right_on = 'DeptID',     # Use a list if 2+ keys
      )
)

LastName,department,DeptName
str,i64,str
"""Rafferty""",31,"""Sales"""
"""Jones""",33,"""Engineering"""
"""Heisenberg""",33,"""Engineering"""
"""Robinson""",34,"""Clerical"""
"""Smith""",34,"""Clerical"""


#### Inner join

In [181]:
(empl
 .join(dept, 
       on = 'DeptID',  # Use a list if 2+ keys
       how = 'inner',
      )
)

LastName,DeptID,DeptName
str,i64,str
"""Rafferty""",31,"""Sales"""
"""Jones""",33,"""Engineering"""
"""Heisenberg""",33,"""Engineering"""
"""Robinson""",34,"""Clerical"""
"""Smith""",34,"""Clerical"""


#### Left join

In [182]:
(empl
 .join(dept, 
       on = 'DeptID',  # Use a list if 2+ keys
       how = 'left',
      )
)

LastName,DeptID,DeptName
str,i64,str
"""Rafferty""",31,"""Sales"""
"""Jones""",33,"""Engineering"""
"""Heisenberg""",33,"""Engineering"""
"""Robinson""",34,"""Clerical"""
"""Smith""",34,"""Clerical"""
"""Williams""",null,null


#### Right join

In [183]:
(empl
 .join(dept, 
       on = 'DeptID',  # Use a list if 2+ keys
       how = 'right',
      )
)

LastName,DeptID,DeptName
str,i64,str
"""Rafferty""",31,"""Sales"""
"""Jones""",33,"""Engineering"""
"""Heisenberg""",33,"""Engineering"""
"""Robinson""",34,"""Clerical"""
"""Smith""",34,"""Clerical"""
null,35,"""Marketing"""


#### Full outer join

In [184]:
(empl
 .join(dept, 
       on = 'DeptID',  # Use a list if 2+ keys
       how = 'full',
      )
)

LastName,DeptID,DeptID_right,DeptName
str,i64,i64,str
"""Rafferty""",31,31,"""Sales"""
"""Jones""",33,33,"""Engineering"""
"""Heisenberg""",33,33,"""Engineering"""
"""Robinson""",34,34,"""Clerical"""
"""Smith""",34,34,"""Clerical"""
"""Williams""",null,null,null
null,null,35,"""Marketing"""


#### Cross

Cross is a cartesian product and will even include all key combinations, even those not present.

In [185]:
(empl
 .join(dept, 
       on = 'DeptID',  # Use a list if 2+ keys
       how = 'cross',
      )
)

LastName,DeptID,DeptID_right,DeptName
str,i64,i64,str
"""Rafferty""",31,31,"""Sales"""
"""Rafferty""",31,33,"""Engineering"""
"""Rafferty""",31,34,"""Clerical"""
"""Rafferty""",31,35,"""Marketing"""
"""Jones""",33,31,"""Sales"""
…,…,…,…
"""Smith""",34,35,"""Marketing"""
"""Williams""",null,31,"""Sales"""
"""Williams""",null,33,"""Engineering"""


#### Anti

An anti join returns all keys in the left table without a match in the right.

In [186]:
(empl
 .join(dept, 
       on = 'DeptID',  # Use a list if 2+ keys
       how = 'anti',
      )
)

LastName,DeptID
str,i64
"""Williams""",null


## Joining on multiple keys

Next, we will look at table joins that require matching multiple keys.

### Example -- Total At Bats, Hits, and Runs Allowed in 2010

To illustrate joining on multiple keys, lets

1. Compute the totals for H and R from 2010+ for each team from the `Pitching` table.
2. Join on the team name and park.

This is a good example, because team information can change over the years, so we need to match both `teamID` and `yearID`.

#### Step 1. Read and process the pitching table

In [187]:
(pitching := 
 pl.read_csv("./data/baseball/core/Pitching.csv", 
            )
).head()

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,f64,str,i64,str,i64,i64,i64,i64,str,str,str
"""bechtge01""",1871,1,"""PH1""","""NA""",1,2,3,3,2,0,0,78,43,23,0,11,1,null,7.96,null,7,null,0,146,0,42,null,null,null
"""brainas01""",1871,1,"""WS3""","""NA""",12,15,30,30,30,0,0,792,361,132,4,37,13,null,4.5,null,7,null,0,1291,0,292,null,null,null
"""fergubo01""",1871,1,"""NY2""","""NA""",0,0,1,0,0,0,0,3,8,3,0,0,0,null,27.0,null,2,null,0,14,0,9,null,null,null
"""fishech01""",1871,1,"""RC1""","""NA""",4,16,24,24,22,1,0,639,295,103,3,31,15,null,4.35,null,20,null,0,1080,1,257,null,null,null
"""fleetfr01""",1871,1,"""NY2""","""NA""",0,1,1,1,1,0,0,27,20,10,0,3,0,null,10.0,null,0,null,0,57,0,21,null,null,null


In [188]:
(teams := 
 pl.read_csv("./data/baseball/core/Teams.csv",
            )
).head()

yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,DivWin,WCWin,LgWin,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,CS,HBP,SF,RA,ER,ERA,CG,SHO,SV,IPouts,HA,HRA,BBA,SOA,E,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
i64,str,str,str,str,i64,i64,str,i64,i64,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,str,str,str,i64,i64,str,str,str
1871,"""NA""","""BS1""","""BNA""",null,3,31,null,20,10,null,null,"""N""",null,401,1372,426,70,37,3,60,19,73,16,null,null,303,109,3.55,22,1,3,828,367,2,42,23,243,24,0.834,"""Boston Red Stockings""","""South End Grounds I""",null,103,98,"""BOS""","""BS1""","""BS1"""
1871,"""NA""","""CH1""","""CNA""",null,2,28,null,19,9,null,null,"""N""",null,302,1196,323,52,21,10,60,22,69,21,null,null,241,77,2.76,25,0,1,753,308,6,28,22,229,16,0.829,"""Chicago White Stockings""","""Union Base-Ball Grounds""",null,104,102,"""CHI""","""CH1""","""CH1"""
1871,"""NA""","""CL1""","""CFC""",null,8,29,null,10,19,null,null,"""N""",null,249,1186,328,35,40,7,26,25,18,8,null,null,341,116,4.11,23,0,0,762,346,13,53,34,234,15,0.818,"""Cleveland Forest Citys""","""National Association Grounds""",null,96,100,"""CLE""","""CL1""","""CL1"""
1871,"""NA""","""FW1""","""KEK""",null,7,19,null,7,12,null,null,"""N""",null,137,746,178,19,8,2,33,9,16,4,null,null,243,97,5.17,19,1,0,507,261,5,21,17,163,8,0.803,"""Fort Wayne Kekiongas""","""Hamilton Field""",null,101,107,"""KEK""","""FW1""","""FW1"""
1871,"""NA""","""NY2""","""NNA""",null,5,33,null,16,17,null,null,"""N""",null,302,1404,403,43,21,1,33,15,46,15,null,null,313,121,3.72,32,1,0,879,373,7,42,22,235,14,0.84,"""New York Mutuals""","""Union Grounds (Brooklyn)""",null,90,88,"""NYU""","""NY2""","""NY2"""


#### Step 2. Process individual tables

In [189]:
(pitching_totals_2010_plus := 
 pitching
 .select('teamID', 'yearID', 'R', 'H')
 .filter(pl.col('yearID') >= 2010)
 .group_by('teamID', 'yearID')
 .agg(pl.sum('R').alias('Total Runs Allowed'),
      pl.sum('H').alias('Total Hits Allowed'),
     )
).head()

teamID,yearID,Total Runs Allowed,Total Hits Allowed
str,i64,i64,i64
"""KCA""",2013,601,1366
"""MIL""",2015,737,1432
"""PIT""",2011,712,1513
"""PIT""",2013,577,1299
"""CIN""",2015,754,1436


In [190]:
(team_name_and_park := 
 teams
 .select('yearID', 
         'teamID',
         pl.col('name').alias('Team Name'),
         pl.col('park').alias('Home Park'),
        )
).head()

yearID,teamID,Team Name,Home Park
i64,str,str,str
1871,"""BS1""","""Boston Red Stockings""","""South End Grounds I"""
1871,"""CH1""","""Chicago White Stockings""","""Union Base-Ball Grounds"""
1871,"""CL1""","""Cleveland Forest Citys""","""National Association Grounds"""
1871,"""FW1""","""Fort Wayne Kekiongas""","""Hamilton Field"""
1871,"""NY2""","""New York Mutuals""","""Union Grounds (Brooklyn)"""


#### Step 3. Perform a left-join.

Since we want to keep all rows in the totals table, and only add the team information when available, we will perform a left join on the totals table.

Notice that 
1. the second `on` argument is now a `list` of column expressions, one for each matching rule, and
2. We have cleaned and reordered the pivot table to make it more readable.

In [191]:
(pitching_totals_2010_plus
 .join(team_name_and_park,
       on = ['yearID', 'teamID'],
       how='left',
       )
 .drop('teamID')
 .select('Team Name',
         pl.col('yearID').alias('Year'),
         'Total Runs Allowed',
         'Total Hits Allowed',
         'Home Park',
        )
).head()

Team Name,Year,Total Runs Allowed,Total Hits Allowed,Home Park
str,i64,i64,i64,str
"""Kansas City Royals""",2013,601,1366,"""Kauffman Stadium"""
"""Milwaukee Brewers""",2015,737,1432,"""Miller Park"""
"""Pittsburgh Pirates""",2011,712,1513,"""PNC Park"""
"""Pittsburgh Pirates""",2013,577,1299,"""PNC Park"""
"""Cincinnati Reds""",2015,754,1436,"""Great American Ball Park"""


## Concatenating Tables with Set-Like Operations in `pyspark`

Now let's look at combining tables with `union`, `intersect`, and `except` in `pyspark`.

 | Option | Advantage | Disadvantage |
 |--------|-----------|--------------|
 | `glob` path | Super easy | No access to file/path |
 | `pl.concat` | Allows pre-processing tables (e.g., add path) | Needs same columns/order; Messy dot-chain |
 | `pl.sql` query | More SQL like | Messy, SQL query in a multiline string | 

#### Method 1 - Built in union with `polars glob` matching 

In [205]:
(sales_glob := 
 pl.read_csv("./data/auto_sales_*.csv",  
            )
)

ID,Salesperson,Compact,Sedan,SUV,Truck
i64,str,i64,i64,i64,i64
0,"""Ann""",22,18,15,12
1,"""Bob""",19,12,17,20
2,"""Yolanda""",19,8,32,15
3,"""Xerxes""",12,23,18,9
0,"""Ann""",22,18,15,12
1,"""Bob""",20,14,6,24
2,"""Yolanda""",19,10,28,17
3,"""Xerxes""",11,27,17,9


#### Method 2 - `pl.concat`

In [206]:
(combined_sales := 
 pl.concat([pl.read_csv("./data/auto_sales_apr.csv")
              .with_columns(Month = pl.lit('apr')),
            pl.read_csv("./data/auto_sales_may.csv")
              .with_columns(Month = pl.lit('may')),
           ])
)

ID,Salesperson,Compact,Sedan,SUV,Truck,Month
i64,str,i64,i64,i64,i64,str
0,"""Ann""",22,18,15,12,"""apr"""
1,"""Bob""",19,12,17,20,"""apr"""
2,"""Yolanda""",19,8,32,15,"""apr"""
3,"""Xerxes""",12,23,18,9,"""apr"""
0,"""Ann""",22,18,15,12,"""may"""
1,"""Bob""",20,14,6,24,"""may"""
2,"""Yolanda""",19,10,28,17,"""may"""
3,"""Xerxes""",11,27,17,9,"""may"""


#### `pl.sql` and UNION and UNION DISTINCT

In [207]:
auto_apr = pl.read_csv("./data/auto_sales_apr.csv")
auto_may = pl.read_csv("./data/auto_sales_may.csv")

In [195]:
(pl.sql("""
 SELECT Salesperson, Compact, Sedan, SUV, Truck, 'apr' as Month
 FROM auto_apr
 UNION
 SELECT Salesperson, Compact, Sedan, SUV, Truck, 'may' as Month
 FROM auto_may
 """)
).collect()

Salesperson,Compact,Sedan,SUV,Truck,Month
str,i64,i64,i64,i64,str
"""Ann""",22,18,15,12,"""may"""
"""Yolanda""",19,10,28,17,"""may"""
"""Yolanda""",19,8,32,15,"""apr"""
"""Xerxes""",11,27,17,9,"""may"""
"""Xerxes""",12,23,18,9,"""apr"""
"""Ann""",22,18,15,12,"""apr"""
"""Bob""",20,14,6,24,"""may"""
"""Bob""",19,12,17,20,"""apr"""


In [208]:
(pl.sql("""
 SELECT Salesperson, Compact, Sedan, SUV, Truck, 'apr' as Month
 FROM auto_apr
 UNION ALL
 SELECT Salesperson, Compact, Sedan, SUV, Truck, 'may' as Month
 FROM auto_may
 """)
).collect()

Salesperson,Compact,Sedan,SUV,Truck,Month
str,i64,i64,i64,i64,str
"""Ann""",22,18,15,12,"""apr"""
"""Bob""",19,12,17,20,"""apr"""
"""Yolanda""",19,8,32,15,"""apr"""
"""Xerxes""",12,23,18,9,"""apr"""
"""Ann""",22,18,15,12,"""may"""
"""Bob""",20,14,6,24,"""may"""
"""Yolanda""",19,10,28,17,"""may"""
"""Xerxes""",11,27,17,9,"""may"""


#### INTERSECTION

In [209]:
(pl.sql("""
 SELECT Salesperson, Compact, Sedan, SUV, Truck, 'apr' as Month
 FROM auto_apr
 INTERSECT
 SELECT Salesperson, Compact, Sedan, SUV, Truck, 'may' as Month
 FROM auto_may
 """)
).collect()

Salesperson,Compact,Sedan,SUV,Truck,Month
str,i64,i64,i64,i64,str


In [210]:
(pl.sql("""
 SELECT Salesperson, Compact, Sedan, SUV, Truck
 FROM auto_apr
 INTERSECT
 SELECT Salesperson, Compact, Sedan, SUV, Truck
 FROM auto_may
 """)
).collect()

Salesperson,Compact,Sedan,SUV,Truck
str,i64,i64,i64,i64
"""Ann""",22,18,15,12


#### DIFFERENCE

In [211]:
(pl.sql("""
 SELECT Salesperson, Compact, Sedan, SUV, Truck
 FROM auto_apr
 EXCEPT
 SELECT Salesperson, Compact, Sedan, SUV, Truck
 FROM auto_may
 """)
).collect()

Salesperson,Compact,Sedan,SUV,Truck
str,i64,i64,i64,i64
"""Yolanda""",19,8,32,15
"""Xerxes""",12,23,18,9
"""Bob""",19,12,17,20


## Reshaping tables

#### Stacking columns with `unpivot`

In [203]:
(stacked_sales :=
 combined_sales
 .unpivot(on = ['Compact','Sedan','SUV','Truck'],
          index = ['Salesperson', 'Month'],
          variable_name='Type',
          value_name='Sales'
         )
).head()

Salesperson,Month,Type,Sales
str,str,str,i64
"""Ann""","""apr""","""Compact""",22
"""Bob""","""apr""","""Compact""",19
"""Yolanda""","""apr""","""Compact""",19
"""Xerxes""","""apr""","""Compact""",12
"""Ann""","""may""","""Compact""",22


#### Unstacking columns with `pivot`

In [204]:
(stacked_sales
 .pivot(on = 'Type',
        index = 'Salesperson',
        values = 'Sales',
        aggregate_function = 'sum',
       )
)

Salesperson,Compact,Sedan,SUV,Truck
str,i64,i64,i64,i64
"""Ann""",44,36,30,24
"""Bob""",39,26,23,44
"""Yolanda""",38,18,60,32
"""Xerxes""",23,50,35,18
